# Bibioltecas

In [1]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import chart_studio.plotly as py
#pd.set_option("max_rows", None)

# Fator Complementar - Abrangência

In [2]:
# Data de hoje
today = datetime.today()
d = today.strftime("%Y%m%d")
data_atual = str(d)

# Importar dados csv de SG
fator_complementar = pd.read_csv(f'./dados/csv/sg-{data_atual}', sep=';')

In [3]:
fator_complementar

,DT_NOTIFIC,PCR_RESUL
0,2020-04-06,4
1,2020-04-06,4
2,2020-04-06,4
3,2020-03-30,4
4,2020-04-03,4
...,...,...
4618,2021-07-08,4
4619,2021-07-08,4
4620,2021-07-08,4
4621,2021-07-09,4


In [4]:
fator_complementar['PCR_RESUL'].unique()

array([4, 1, 2, 3])

In [5]:
# Criar dataframe com todas as datas do último ano
ano = pd.Series(pd.date_range(end=data_atual, periods=365, freq='D', name="DT_NOTIFIC"))
abrangencia = ano.to_frame()

In [6]:
abrangencia['REALIZADO'] = 0
abrangencia['NAO_REALIZADO'] = 0
abrangencia['NAO_REALIZADO'] = abrangencia['NAO_REALIZADO'].astype(int)
abrangencia['REALIZADO'] = abrangencia['REALIZADO'].astype(int)
abrangencia['DT_NOTIFIC'] = pd.to_datetime(abrangencia['DT_NOTIFIC']).dt.date
abrangencia.set_index('DT_NOTIFIC', inplace = True)

In [7]:
abrangencia

,REALIZADO,NAO_REALIZADO
DT_NOTIFIC,,
2020-07-10,0,0
2020-07-11,0,0
2020-07-12,0,0
2020-07-13,0,0
2020-07-14,0,0
...,...,...
2021-07-05,0,0
2021-07-06,0,0
2021-07-07,0,0


In [8]:
# RT-PCR NAO REALIZADO - Ou seja, foi realizado outro tipo de testagem
filtrados = fator_complementar['PCR_RESUL']==4
nao_realizado = fator_complementar[filtrados]
nao_realizado = nao_realizado.groupby('DT_NOTIFIC').sum()/4

nao_realizado.rename(columns={'PCR_RESUL': 'NAO_REALIZADO'}, inplace = True)
nao_realizado['NAO_REALIZADO'] = nao_realizado['NAO_REALIZADO'].astype(int)
#nao_realizado = nao_realizado.reset_index()

In [9]:
nao_realizado

,NAO_REALIZADO
DT_NOTIFIC,
2020-02-03,2
2020-02-04,2
2020-02-13,2
2020-02-22,1
2020-02-25,1
...,...
2021-07-05,9
2021-07-06,5
2021-07-07,3


In [10]:
# RT-PCR REALIZADO
filtrados = fator_complementar['PCR_RESUL']!=4
realizado = fator_complementar[filtrados]
# Transformando diferentes resutlados de teste realizado em 1 para somar depois
realizado['PCR_RESUL'] = realizado['PCR_RESUL'].map({1:1, 2:1,3:1,5:1})
# Agrupando por data e somando casos negativos
realizado = realizado.groupby('DT_NOTIFIC').sum()
realizado.rename(columns={'PCR_RESUL': 'REALIZADO'}, inplace = True)
#realizado = realizado.reset_index()
realizado

<ipython-input-10-aa4846a4de1b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  realizado['PCR_RESUL'] = realizado['PCR_RESUL'].map({1:1, 2:1,3:1,5:1})


,REALIZADO
DT_NOTIFIC,
2020-02-18,5
2020-03-02,7
2020-03-11,1
2020-03-13,1
2020-03-16,2
...,...
2021-07-01,2
2021-07-05,5
2021-07-06,2


In [11]:
realizado['REALIZADO'].sum()

874

In [12]:
# Juntando realizados e nao realizados
abrangencia = pd.concat([abrangencia, realizado, nao_realizado])

In [13]:
# Agrupando por data
abrangencia = abrangencia.groupby('DT_NOTIFIC').sum()

In [14]:
abrangencia

,REALIZADO,NAO_REALIZADO
DT_NOTIFIC,,
2020-07-10,0.0,0.0
2020-07-11,0.0,0.0
2020-07-12,0.0,0.0
2020-07-13,0.0,0.0
2020-07-14,0.0,0.0
...,...,...
2021-07-05,5.0,9.0
2021-07-06,2.0,5.0
2021-07-07,1.0,3.0


In [15]:
#  Segundo o documento da FIOCRUZ abaixo, é  'importante ter uma cobertura de testes para no mínimo 80% da população com Síndrome Gripal.''
# No caso, estamos calculando 80% apenas para SG
# https://portal.fiocruz.br/sites/portal.fiocruz.br/files/documentos/contribuicoes_para_o_retorno_escolar_17122020.pdf

In [16]:
# Acumulativo dos últimos 7 dias
rolling = abrangencia.rolling(window=7).sum()

In [17]:
# Porcentagem média movel 7 dias
rolling['PORCENTAGEM MEDIA MOVEL 7 DIAS'] = rolling['REALIZADO']*100/(rolling['REALIZADO'] + rolling['NAO_REALIZADO'])

In [18]:
rolling

,REALIZADO,NAO_REALIZADO,PORCENTAGEM MEDIA MOVEL 7 DIAS
DT_NOTIFIC,,,
2020-07-10,NaN,NaN,NaN
2020-07-11,NaN,NaN,NaN
2020-07-12,NaN,NaN,NaN
2020-07-13,NaN,NaN,NaN
2020-07-14,NaN,NaN,NaN
...,...,...,...
2021-07-05,8.0,38.0,17.391304
2021-07-06,9.0,34.0,20.930233
2021-07-07,10.0,35.0,22.222222


In [19]:
rolling.to_csv(f'./dados/csv/fator-abrangencia-{data_atual}', sep=';')

In [20]:
abrangencia = pd.read_csv(f'./dados/csv/fator-abrangencia-{data_atual}', sep=';')

In [21]:
abrangencia = abrangencia[['DT_NOTIFIC','PORCENTAGEM MEDIA MOVEL 7 DIAS']]

In [22]:
abrangencia

,DT_NOTIFIC,PORCENTAGEM MEDIA MOVEL 7 DIAS
0,2020-07-10,NaN
1,2020-07-11,NaN
2,2020-07-12,NaN
3,2020-07-13,NaN
4,2020-07-14,NaN
...,...,...
822,2021-07-05,17.391304
823,2021-07-06,20.930233
824,2021-07-07,22.222222
825,2021-07-08,14.814815


In [23]:
# Retirar avalores NaN e colocar o campo "PORCENTAGEM MEDIA MOVEL 7 DIAS" como inteiro
abrangencia = abrangencia.dropna()
abrangencia["PORCENTAGEM MEDIA MOVEL 7 DIAS"] = abrangencia["PORCENTAGEM MEDIA MOVEL 7 DIAS"].astype(int)
abrangencia = abrangencia.rename(columns={'DT_NOTIFIC': 'Data','PORCENTAGEM MEDIA MOVEL 7 DIAS':'Abrangencia'})

<ipython-input-23-10e79203da81>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abrangencia["PORCENTAGEM MEDIA MOVEL 7 DIAS"] = abrangencia["PORCENTAGEM MEDIA MOVEL 7 DIAS"].astype(int)


In [24]:
abrangencia

,Data,Abrangencia
365,2020-02-03,0
366,2020-02-04,0
367,2020-02-13,0
368,2020-02-18,45
369,2020-02-22,41
...,...,...
822,2021-07-05,17
823,2021-07-06,20
824,2021-07-07,22
825,2021-07-08,14


In [25]:
abrangencia = abrangencia.tail(365).reset_index(drop=True)
abrangencia

,Data,Abrangencia
0,2020-06-26,10
1,2020-06-27,6
2,2020-06-28,8
3,2020-06-29,11
4,2020-06-30,13
...,...,...
360,2021-07-05,17
361,2021-07-06,20
362,2021-07-07,22
363,2021-07-08,14


In [27]:
hoje = today.strftime("%d-%m-%Y")

fig = px.line(abrangencia, x="Data", y="Abrangencia", title=f'Abrangência de testes RT-PCR realizados em casos de Síndrome Gripal (média móvel de 7 dias) - Atualizado em {hoje}', labels=dict(Porcentagem="Porcentagem",Data="" ))

fig.add_hrect(y0=80, y1=100, line_width=0, fillcolor="green", opacity=0.2)
fig.add_hrect(y0=0, y1=80, line_width=0, fillcolor="red", opacity=0.2)



fig.update_yaxes(range=[0,100], fixedrange= True) # linear range
fig.update_xaxes(fixedrange= True)

fig.show()
fig.write_html("indicador-2.html")


py.sign_in(usuario, senha)
plot_url = py.plot(fig)